In [1]:
import pandas as pd
from selenium import webdriver
import time
from tqdm import tqdm

In [2]:
train = pd.read_csv("./data/train.csv")

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(chrome_options=options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [4]:
def trans(t, driver):
    new_text = []
    lang = ["ko", 'ja&hn=0']
    for l in lang:
        backtrans = ""
        try:
            driver.get('https://papago.naver.com/?sk=en&tk='+l+'&st='+t)
            time.sleep(2.5)
            backtrans = driver.find_element_by_xpath('//*[@id="txtTarget"]').text
        except:
            pass
        if backtrans != "":
            return_sen = ""
            try:
                driver.get('https://papago.naver.com/?sk='+l+'&tk=en&st='+backtrans)
                time.sleep(2.5)
                return_sen = driver.find_element_by_xpath('//*[@id="txtTarget"]').text
            except:
                pass
            if return_sen != "":
                new_text.append(return_sen)
    return list(set(new_text))


In [5]:
def backtranslation(driver):
    train = pd.read_csv("./data/train.csv")

    aug_label = []
    val = train["Target"].values.tolist()
    val = list(set(val))

    for v in val:
        if len(train[train["Target"] == v]) < 1300:
            aug_label.append(v)

    aug_data = pd.DataFrame()
    for a in tqdm(aug_label):
        aug_data = aug_data.append(train[train["Target"] == a])
    
    auged_data = pd.DataFrame()
    for idx in tqdm(range(len(aug_data))):
        original = aug_data.iloc[idx]
        new_text = trans(original["Utterance"], driver)
        for n in new_text:
            if original["Utterance"] != n:
                add = pd.DataFrame([{
                    "ID" : original["ID"],
                    "Utterance" : n,
                    "Speaker" : original["Speaker"],
                    "Dialogue_ID" : original["Dialogue_ID"],
                    "Target": original["Target"]
                }])
                auged_data = auged_data.append(add)
    train = train.append(auged_data)
    train.to_csv("./data/train_backtrans.csv")


In [6]:
backtranslation(driver)

100%|██████████| 3536/3536 [10:26:42<00:00, 10.63s/it] 
